# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import base64
from config import Config
from datetime import datetime, timedelta
import os
import requests
from IPython import get_ipython
from IPython.display import display, Markdown

%load_ext sql
%config SqlMagic.displaylimit = 5

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
set time zone 'UTC';

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT 
    id, 
    (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date,  
    total,  
    service_fees, 
    payment_method_code, 
    corp_type_code, 
    created_by, 
    payment_date
FROM 
    invoices
WHERE 
    corp_type_code = :partner_code
    AND total > 0
    AND invoice_status_code = 'PAID'
    AND payment_method_code in ('PAD','EJV')
    AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  > (current_date - 1 - interval '1 months')::date
    AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver' <= (current_date - 1)::date
ORDER BY 
    1;

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

disbursement summary

In [ ]:
# Define the partner codes for which the disbursement summary should be executed
partners_for_disbursement_summary = ['CSO', 'VS']

# Print the partner_code and the list to confirm their values
print(f"Processing for partner_code: {partner_code}")
print(f"Partners for disbursement summary: {partners_for_disbursement_summary}")

if partner_code in partners_for_disbursement_summary:
    print(f"Partner code {partner_code} found in the list, executing SQL query.")
    query = f"""
    SELECT id, (disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, total, service_fees, payment_method_code, corp_type_code,created_by
    FROM invoices
    WHERE corp_type_code = :partner_code
    AND invoice_status_code = 'PAID'
    AND payment_method_code in ('PAD','EJV')
    AND disbursement_status_code = 'COMPLETED'
    AND disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  > (current_date - 1 - interval '1 months'- interval '5 days')::date
    AND disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  <= (current_date - 1)::date
    order by 1;
    """
    
    display(Markdown(f"## Running query for partner: {partner_code}"))
    results = get_ipython().run_cell_magic('sql', '', query)
    monthly_reconciliation_disbursed = results.DataFrame()  # Convert the results to a DataFrame for later use
else:
    print(f"Partner code {partner_code} not in the list, skipping SQL query.")
    monthly_reconciliation_disbursed = None  # Assign None if not executed

Save to another CSV

In [ ]:
if monthly_reconciliation_disbursed is not None and not monthly_reconciliation_disbursed.empty:
    filename_disbursed = os.path.join(os.getcwd(), 'data', f'{partner_code}_monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d') + '.csv')
    print(f"Saving CSV for partner_code: {partner_code} at {filename_disbursed}")
    with open(filename_disbursed, 'w') as f:
        f.write('Monthly Reconciliation Disbursed:\n\n')
        if monthly_reconciliation_disbursed.empty:
            f.write('No Data Retrieved')
        else:
            monthly_reconciliation_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)
else:
    print(f"Partner code {partner_code} not in the list or no data to save, skipping CSV save.")


Authenticate

In [ ]:
payload = "grant_type=client_credentials"
basic_hash = base64.b64encode(f"{os.getenv('NOTEBOOK_SERVICE_ACCOUNT_ID')}:{os.getenv('NOTEBOOK_SERVICE_ACCOUNT_SECRET')}".encode())
    
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': f'Basic {basic_hash.decode()}'
}
response = requests.request("POST", f"{os.getenv('JWT_OIDC_ISSUER')}/protocol/openid-connect/token", headers=headers, data=payload)

assert response.status_code == 200
notebook_service_account_token = response.json().get('access_token')

In [ ]:

partner_details = {
    'CSO': {
        "companyName": "Ministry of Justice",
        "addressLine1": "PO Box 9249, Stn Prov Govt",
        "addressLine2": "6th Floor, 850 Burdett Avenue",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9J2"
    },
    'VS': {
        "companyName": "Vital Statistics Agency",
        "addressLine1": "PO Box 9657, Stn Prov Govt",
        "addressLine2": "",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9P3"
    },
    'RPT': {
        "companyName": "Property Taxation Branch",
        "addressLine1": "Ministry of Provincial Revenue",
        "addressLine2": "4th Floor, 1802 Douglas Street",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8T 4K6"
    },
    'ESRA': {
        "companyName": "Ministry of Enviornment",
        "addressLine1": "Environmental Emergencies and Land Remediation",
        "addressLine2": "2nd Floorm 10470-152nd Street",
        "city": "SURREY",
        "province": "BC",
        "areaCode": "V3R 0Y3"
    }
    
}

def generate_report(partner_code):
    # Get the API base URL from the environment variable
    API_BASE_URL = os.getenv('REPORT_API_URL', '')
    if not API_BASE_URL:
        raise ValueError("The REPORT_API_URL environment variable is not set or is empty")

    url = API_BASE_URL
    headers = {
        'Authorization': f'Bearer {notebook_service_account_token}',
        'Content-Type': 'application/json',
        'Accept': 'application/pdf'
    }

    # SQL query
    query = """
    SELECT
        COUNT(*) AS transaction_count,
        SUM(total) AS total,
        TO_CHAR(DATE_TRUNC('month',current_date) - INTERVAL '1 month','Month') as month,
        corp_type_code
    FROM 
        invoices
    WHERE 
        corp_type_code = :partner_code
        AND invoice_status_code = 'PAID'
        AND payment_method_code IN ('PAD', 'EJV')
        AND DATE_TRUNC('month', created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') = DATE_TRUNC('month', current_date - INTERVAL '1 month')
    GROUP BY 
        corp_type_code
    ORDER BY 
        month;
    """

    # Execute the SQL query and fetch results
    result = %sql $query

    # Print the result to verify
    print(result)

    # Check if the result is not None
    if result:
        # Convert the result to a DataFrame
        df = result.DataFrame()

        # Rename columns to match the expected names in the template
        df.rename(columns={
            'corp_type_code': 'registry',
            'transaction_count': 'transCounts',
            'total': 'amount'
        }, inplace=True)

        # Add a Symbol column with a dollar sign
        df['symbol'] = '$'

        # Convert DataFrame to JSON-compatible format
        tableRows = df.to_dict(orient='records')

        # Get the current date
        current_date = datetime.now().strftime("%B %d, %Y")
        
        # Get the details for the current partner
        details = partner_details.get(partner_code, {})
        if not details:
            raise ValueError(f"No details found for partner code: {partner_code}")

        # Define the request body
        data = {
            "templateVars": {
                "date": current_date,
                "companyName": details["companyName"],
                "addressLine1": details["addressLine1"],
                "addressLine2": details["addressLine2"],
                "city": details["city"],
                "province": details["province"],
                "areaCode": details["areaCode"],
                "firstName": partner_code,
                "enterMonth": df['month'][0] if not df.empty else "N/A",
                "tableRows": tableRows
            },
            "templateName": "revenue_letter",
            "reportName": "revenue_letter"
        }

        # Send the POST request
        response = requests.post(url, headers=headers, json=data)

        # Check if the response is successful
        if response.status_code == 200:
            # Get the PDF content from the response
            pdf_content = response.content
            
            # Save the PDF content to a file
            pdf_filename = os.path.join(os.getcwd(), 'data', f'{partner_code}_revenue_letter.pdf')
            with open(pdf_filename, 'wb') as pdf_file:
                pdf_file.write(pdf_content)
            
            print("PDF report saved successfully as 'payment_receipt.pdf'")
        else:
            print('Failed to get the report:', response.text)
    else:
        print('No results returned from the SQL query')
generate_report(partner_code)